In [1]:
from neon.layers import GeneralizedCost, LSTM, Affine
from neon.models import Model
from neon.optimizers import RMSProp
from neon.transforms import Logistic, Tanh, Softmax, CrossEntropyMulti
from neon.callbacks.callbacks import Callbacks
from neon.util.argparser import NeonArgparser
from neon.data import Text
from neon.initializers import GlorotUniform, Uniform, Orthonormal

In [2]:
import numpy as np
import os
import datetime
import cPickle
import sklearn
from sklearn.cross_validation import train_test_split

In [3]:
from neon.backends import gen_backend
be = gen_backend(backend='cpu',batch_size=10)
rnn_size = 256

In [33]:
f = open('data.pkl', 'r')

classes = cPickle.load( f )
chars = cPickle.load( f )
char_indices = cPickle.load( f )
indices_char = cPickle.load( f )

maxlen = cPickle.load( f )
step = cPickle.load( f )

X_ind = cPickle.load( f )
y_ind = cPickle.load( f )

f.close()

[ s1, s2 ] = X_ind.shape

X = np.zeros( ( s1, s2, len( chars ) ), dtype=np.bool )
y = np.zeros( ( s1, len( chars ) ), dtype=np.bool )

for i in range( s1 ):
    for t in range( s2 ):
        X[ i, t, X_ind[ i, t ] ] = 1
y[ i, y_ind[ i ] ] = 1

In [34]:
print y.shape
print X.shape

(3066887, 99)
(3066887, 20, 99)


In [6]:
X = X.reshape((3066887, 20*99))
print y.shape
print X.shape
# X_train, y_train, X_test, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

(3066887, 99)
(3066887, 1980)


In [7]:
from random import sample
l = X.shape[0]
f = int(X.shape[0] * 0.8)
indices = sample(range(l), f)
X_train = X[indices]
X_test = np.delete(X, indices, axis=0)
y_train = y[indices]
y_test = np.delete(y, indices, axis=0)
print X_train.shape
print y_train.shape

(2453509, 1980)
(2453509, 99)


In [8]:
from neon.data import ArrayIterator
train = ArrayIterator(X=X_train, y=y_train, lshape = (1, 99, 20), make_onehot=False)


In [14]:
valid = ArrayIterator(X=X_test, y=y_test, make_onehot=False)

In [26]:
init_glorot = GlorotUniform()
init_uniform = Uniform(-0.1/128, 0.1/128)
init_orthonormal = Orthonormal()

In [27]:
layers = [LSTM(output_size=rnn_size, init=init_glorot, activation=Tanh(),gate_activation=Tanh(),
               init_inner=init_orthonormal),
         Affine(len(chars), init=init_glorot, activation=Softmax())]

In [28]:
optimizer = RMSProp()
cost = GeneralizedCost(costfunc=CrossEntropyMulti(usebits=True))

In [29]:
from neon.callbacks import Callbacks
from neon.models import Model
model = Model(layers=layers)
num_epochs = 2
fname = "neon_lstm_model"
model = Model(layers=layers)
callbacks = Callbacks(model, eval_set=valid, eval_freq=num_epochs,
                      serialize=1, save_path=fname + '.pickle')

In [30]:
model.fit(train, optimizer=optimizer, num_epochs=64, cost=cost, callbacks=callbacks)

ValueError: CrossEntropy requires network output shape to match targets. Network output shape was (99, 400) and targets shape was (99, 20)

In [67]:
a = np.arange(60).reshape((10, 3, 2))